In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import fileio
import keys
import evaluation
import full_model
import data_aug
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import h5py

In [2]:
DATA_DIR = './data/working'

labels = pd.read_pickle("{}/labels.pkl".format(DATA_DIR))

with np.load("{}/splits.npz".format(DATA_DIR)) as splits:
    train_idx = splits['train_idx']
    test_idx = splits['test_idx']

X = np.load("{}/X_cqt.npz".format(DATA_DIR))['X']
X_train = X[train_idx, :]
X_test = X[test_idx, :]

Y = np.load("{}/Y.npz".format(DATA_DIR))['Y']
Y_train = Y[train_idx, :]
Y_test = Y[test_idx, :]

aug = np.load("{}/data_aug.npz".format(DATA_DIR))

with np.load("{}/data_aug.npz".format(DATA_DIR)) as aug:
    X_train = np.vstack((X_train, aug['X']))
    Y_train = np.vstack((Y_train, aug['Y']))

X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()

Y_train = torch.from_numpy(np.argmax(Y_train, axis=1))
Y_test = torch.from_numpy(np.argmax(Y_test, axis=1))

In [3]:
import full_model
import model_tester
import importlib
importlib.reload(full_model)
importlib.reload(model_tester)

h5_file = 'data/working/data.h5'

os.environ["CUDA_VISIBLE_DEVICES"]="2"
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

losses = model_tester.train_model(h5_file, full_model.ConvBiLstm, weight_decay=0.0005,
                         device=device, batch_size=1)

ConvBiLstm(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(10, 10, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(10, 10, kernel_size=(10, 10), stride=(2, 2), padding=(4, 4))
  (dense): Conv2d(10, 48, kernel_size=(72, 1), stride=(1, 1))
  (lstm): GRU(48, 48, num_layers=2, batch_first=True, bidirectional=True)
  (conv_pool): Conv2d(1, 10, kernel_size=(1, 75), stride=(1, 1))
  (fc1): Linear(in_features=96, out_features=48, bias=True)
  (fc3): Linear(in_features=48, out_features=24, bias=True)
)
Batch 0/12703


RuntimeError: dimension out of range (expected to be in range of [-2, 1], but got 2)

In [170]:
DATA_DIR = './data/working'

labels = pd.read_pickle("{}/labels.pkl".format(DATA_DIR))

with np.load("{}/splits.npz".format(DATA_DIR)) as splits:
    train_idx = splits['train_idx']
    test_idx = splits['test_idx']
    
X = np.load("{}/X_cqt.npz".format(DATA_DIR))['X']
X_train = X[train_idx, :]
X_test = X[test_idx, :]

Y = np.load("{}/Y.npz".format(DATA_DIR))['Y']
Y_train = Y[train_idx, :]
Y_test = Y[test_idx, :]

In [12]:
import h5py

file = h5py.File('data/working/data.h5', 'w')
file.create_dataset('X_train', data=X_train, dtype=float, compression='lzf')
file.create_dataset('Y_train', data=Y_train, dtype='i8', compression='lzf')
file.create_dataset('X_test', data=X_test, dtype=float, compression='lzf')
file.create_dataset('Y_test', data=Y_test, dtype='i8', compression='lzf')

file.close()

In [205]:
file = h5py.File('data/working/data.h5', 'r')
file['X_train'].shape[0]
file.close()

In [206]:
X_test.shape

(4227, 144, 151)